## Overview

This notebook provides a comprehensive demonstration of several baseline algorithms used in First-Price Auction (FPA) and VCG (Vickrey–Clarke–Groves) auction scenarios. We showcase both individual performances and comparative analyses of the following algorithms:

1. ALM
2. TA-PID
3. M-PID
4. Mystique
5. BROI

## Purpose

The main objectives of this notebook are:

1. To illustrate the implementation and behavior of each baseline algorithm
2. To compare the performance of these algorithms

Feel free to add your own bidding algorithms to our benchmark 😊


# Baselines comparison

In [ ]:
import sys
sys.path.append("../")

%load_ext autoreload
%autoreload 2
    
import pandas as pd
import numpy as np
import optuna
import pickle
import matplotlib.pyplot as plt

from functools import partial

from simulator.simulation.modules import Campaign
from simulator.simulation.utils_visualization import data_prep_vis, plot_history_article
from simulator.simulation.simulate import simulate_campaign
from simulator.validation.check_results import autobidder_check

# import baselines:
from simulator.model.linear_bidder import LinearBidder
from simulator.model.ta_pid import TAPIDBidder
from simulator.model.m_pid import MPIDBidder
from simulator.model.mystique import Mystique
from simulator.model.broi_bidder import BROI
from simulator.model.traffic import Traffic
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', None)

# Load data

In [ ]:
auction_mode = 'FPA' # you may also choose the 'VCG' mode

# data paths
campaigns_path = f"../../data/{auction_mode.lower()}/campaigns_{auction_mode.lower()}_filtered_train_final.csv"
stats_path = f"../data/{auction_mode.lower()}/stats_{auction_mode.lower()}_filtered_train_final.csv"

In [ ]:
campaign_df = pd.read_csv(campaigns_path)
stats_df = pd.read_csv(stats_path)

In [ ]:
campaign_df.sample(1)

In [ ]:
stats_df.sample(3)

# One campaign's example

In [ ]:
campaign_data = campaign_df.sample(1).iloc[0]
campaign_stats = stats_df[stats_df.campaign_id == campaign_data["campaign_id"]].copy()
print(campaign_data.shape, campaign_stats.shape)

## Linear Bidder

In [ ]:
campaign_example = Campaign(
    item_id= campaign_data['item_id'],
    campaign_id=int(campaign_data['campaign_id']),
    loc_id=int(campaign_data["loc_id"]),
    region_id=int(campaign_data["region_id"]),
    logical_category=campaign_data["logical_category"],
    microcat_ext=int(campaign_data["microcat_ext"]),
    campaign_start=int(campaign_data["campaign_start"]),
    campaign_end=int(campaign_data["campaign_end"]),
    initial_balance=campaign_data['auction_budget'],
    balance=campaign_data['auction_budget'],
    curr_time=int(campaign_data["campaign_start"]),
    prev_time=int(campaign_data["campaign_start"]),
    prev_balance=campaign_data['auction_budget'],
)
print("Budget ", campaign_data['auction_budget']) 
print("Length hours ", (campaign_data['campaign_end'] - campaign_data['campaign_start']) // 3600)

In [ ]:
simulation_history = simulate_campaign(
    campaign=campaign_example,
    bidder=LinearBidder(),
    stats_file=campaign_stats,
    auction_mode=auction_mode
)

In [ ]:
df = simulation_history.to_data_frame()
viz_data_linear = data_prep_vis(df)
df.head(2)

In [ ]:
fig = plot_history_article(viz_data_linear)

### BROI

In [ ]:
campaign_example = Campaign(
    item_id= campaign_data['item_id'],
    campaign_id=int(campaign_data['campaign_id']),
    loc_id=int(campaign_data["loc_id"]),
    region_id=int(campaign_data["region_id"]),
    logical_category=campaign_data["logical_category"],
    microcat_ext=int(campaign_data["microcat_ext"]),
    campaign_start=int(campaign_data["campaign_start"]),
    campaign_end=int(campaign_data["campaign_end"]),
    initial_balance=campaign_data['auction_budget'],
    balance=campaign_data['auction_budget'],
    curr_time=int(campaign_data["campaign_start"]),
    prev_time=int(campaign_data["campaign_start"]),
    prev_balance=campaign_data['auction_budget'],
)
print("Budget ", campaign_data['auction_budget']) 
print("Length hours ", (campaign_data['campaign_end'] - campaign_data['campaign_start']) // 3600)

In [ ]:
simulation_history = simulate_campaign(
    campaign=campaign_example,
    bidder=BROI(),
    stats_file=campaign_stats,
    auction_mode=auction_mode
)

In [ ]:
df = simulation_history.to_data_frame()
viz_data_broi = data_prep_vis(df)
df.head(2)

In [ ]:
fig = plot_history_article(viz_data_broi)

## TA-PID Bidder

In [ ]:
# refreshing Campaign
campaign_example = Campaign(
    item_id=campaign_data['item_id'],
    campaign_id=int(campaign_data['campaign_id']),
    loc_id=int(campaign_data["loc_id"]),
    region_id=int(campaign_data["region_id"]),
    logical_category=campaign_data["logical_category"],
    microcat_ext=int(campaign_data["microcat_ext"]),
    campaign_start=int(campaign_data["campaign_start"]),
    campaign_end=int(campaign_data["campaign_end"]),
    initial_balance=campaign_data['auction_budget'],
    balance=campaign_data['auction_budget'],
    curr_time=int(campaign_data["campaign_start"]),
    prev_time=int(campaign_data["campaign_start"]),
    prev_balance=campaign_data['auction_budget'],
)
print("Budget ", campaign_data['auction_budget']) 
print("Length hours ", (campaign_data['campaign_end'] - campaign_data['campaign_start']) // 3600)

In [ ]:
simulation_history = simulate_campaign(
    campaign=campaign_example,
    bidder=TAPIDBidder(),
    stats_file=campaign_stats,
    auction_mode=auction_mode
)

In [ ]:
df = simulation_history.to_data_frame()
viz_data_ta_pid = data_prep_vis(df)
df.head(2)

In [ ]:
fig = plot_history_article(viz_data_ta_pid)

## M-PID Bidder

In [ ]:
# refreshing Campaign
campaign_example = Campaign(
    item_id=campaign_data['item_id'],
    campaign_id=int(campaign_data['campaign_id']),
    loc_id=int(campaign_data["loc_id"]),
    region_id=int(campaign_data["region_id"]),
    logical_category=campaign_data["logical_category"],
    microcat_ext=int(campaign_data["microcat_ext"]),
    campaign_start=int(campaign_data["campaign_start"]),
    campaign_end=int(campaign_data["campaign_end"]),
    initial_balance=campaign_data['auction_budget'],
    balance=campaign_data['auction_budget'],
    curr_time=int(campaign_data["campaign_start"]),
    prev_time=int(campaign_data["campaign_start"]),
    prev_balance=campaign_data['auction_budget'],
)
print("Budget ", campaign_data['auction_budget']) 
print("Length hours ", (campaign_data['campaign_end'] - campaign_data['campaign_start']) // 3600)

In [ ]:
simulation_history = simulate_campaign(
    campaign=campaign_example,
    bidder=MPIDBidder(),
    stats_file=campaign_stats,
    auction_mode=auction_mode,
)

In [ ]:
df = simulation_history.to_data_frame()
viz_data_m_pid = data_prep_vis(df)
df.head(10)

In [ ]:
fig = plot_history_article(viz_data_m_pid)

In [ ]:
# refreshing Campaign
campaign_example = Campaign(
    item_id=campaign_data['item_id'],
    campaign_id=int(campaign_data['campaign_id']),
    loc_id=int(campaign_data["loc_id"]),
    region_id=int(campaign_data["region_id"]),
    logical_category=campaign_data["logical_category"],
    microcat_ext=int(campaign_data["microcat_ext"]),
    campaign_start=int(campaign_data["campaign_start"]),
    campaign_end=int(campaign_data["campaign_end"]),
    initial_balance=campaign_data['auction_budget'],
    balance=campaign_data['auction_budget'],
    curr_time=int(campaign_data["campaign_start"]),
    prev_time=int(campaign_data["campaign_start"]),
    prev_balance=campaign_data['auction_budget'],
)
print("Budget ", campaign_data['auction_budget']) 
print("Length hours ", (campaign_data['campaign_end'] - campaign_data['campaign_start']) // 3600)

In [ ]:
simulation_history = simulate_campaign(
    campaign=campaign_example,
    bidder=BROI(),
    stats_file=campaign_stats,
    auction_mode=auction_mode,
)

In [ ]:
df = simulation_history.to_data_frame()
viz_data_broi = data_prep_vis(df)
df.head(10)

In [ ]:
fig = plot_history_article(viz_data_broi)

## You may also compare the performance of all the baselines:

In [ ]:
f = plot_history_article([viz_data_linear, viz_data_ta_pid, viz_data_m_pid], single_plot=False)

# Metrics

In [ ]:
best_metrics = np.ones((3, 3)) * 100000

In [ ]:
best_metrics

### Mean 

## Linear Bidder

In [ ]:
def objective_linear(trial, metric='RMSE_T', auction_mode='FPA'):
    coef = trial.suggest_float('coef', 0.001, 0.999, log=True) # Cold start coefficient
    lower_clip = trial.suggest_int('lower_clip', 1, 20, log=True)
    upper_clip = trial.suggest_int('upper_clip', 1, 20, log=True)
    factor = trial.suggest_float('factor', 1.1, 10.0, log=True)
   
    res = autobidder_check(
        bidder=LinearBidder,
        params={
            "input_campaigns": campaigns_path,
            "input_stats": stats_path,
            "cold_start_coef": coef,
            "lower_clip": lower_clip,
            "upper_clip": upper_clip,
            "factor": factor # slope
        },
        auction_mode=auction_mode,
    )
    print(f"CPC_REL: {res['score'][0]}, rmse: {res['score'][1]}, SCR: {res['score'][2]}")
    if metric == 'RMSE_T':
        best_metrics[0, 0] = min(best_metrics[0, 0], res['score'][1])
        return res['score'][1] 
    elif metric == 'CPC_REL':
        best_metrics[0, 1] = min(best_metrics[0, 1], res['score'][0])
      
        return res['score'][0] 
    elif metric == 'MCR':
        best_metrics[0, 2] = max(best_metrics[0, 2], res['score'][2])
        return res['score'][2] 

    
def opt_search_linear(n_trials=100, metric='RMSE_T', auction_mode='FPA'):
    # Create pruner
    pruner = optuna.pruners.MedianPruner(
        n_startup_trials=5,
        n_warmup_steps=10,
        interval_steps=1
    )

    # Create study with pruner
    study = optuna.create_study(
        direction='maximize' if (metric == 'MCR') else 'minimize',
        pruner=pruner
    )

    # Optimization
    study.optimize(
        partial(objective_linear, metric=metric, auction_mode=auction_mode),
        n_trials=n_trials,
        catch=(Exception,)
    )

    print('Best trial:')
    trial = study.best_trial
    print(f'  Value: {trial.value}')
    print('  Params: ')
    dict_path = f'best_params/linear_{metric.lower()}_{auction_mode}.pkl'
    params_dict = dict()
    for key, value in trial.params.items():
        print(f'    {key}: {value}')
        params_dict[key] = value
    # save the best params to the file
    with open(dict_path, 'wb') as f:
        pickle.dump(params_dict, f)
    return study

In [ ]:
n = 10
metric = 'RMSE_T' # for Mean Click Ratio use 'MCR'
study_linear = opt_search_linear(n, metric, auction_mode)

In [ ]:
n = 10
metric = 'CPC_REL' # for Mean Click Ratio use 'MCR'
study_linear = opt_search_linear(n, metric, auction_mode)

In [ ]:
n = 10
metric = 'MCR' # for Mean Click Ratio use 'MCR'
study_linear = opt_search_linear(n, metric, auction_mode)

## TA-PID Bidder

In [ ]:
def objective_tapid(trial, metric='RMSE_T', auction_mode='FPA'):
    k_p1 = trial.suggest_float('k_p1', 1e-4, 1, log=True)
    k_i1 = trial.suggest_float('k_i1', 1e-4, 1, log=True)
    k_d1 = trial.suggest_float('k_d1', 1e-4, 1, log=True)
    coef = trial.suggest_float('coef', 0.1, 0.9, log=True) # Cold start coefficient
    k_dict = {
        'k_p': k_p1,
        'k_i': k_i1,
        'k_d': k_d1
    }
   
    res = autobidder_check(
        bidder=TAPIDBidder,
        params={
            "input_campaigns": campaigns_path,
            "input_stats": stats_path,
            "k_dict": k_dict, 
            'cold_start_coef': coef,
        },
        auction_mode=auction_mode,
    )
    print(f"CPC_REL: {res['score'][0]}, rmse: {res['score'][1]}, SCR: {res['score'][2]}")
    if metric == 'RMSE_T':
        best_metrics[0, 0] = min(best_metrics[0, 0], res['score'][1])
        return res['score'][1] 
    elif metric == 'CPC_REL':
        best_metrics[0, 1] = min(best_metrics[0, 1], res['score'][0])
        print(f'PROBLEMS???{abs(1 - res["score"][0])}')
        return res['score'][0] 
    elif metric == 'MCR':
        best_metrics[0, 2] = max(best_metrics[0, 2], res['score'][2])
        return res['score'][2] 

def opt_search_tapid(n_trials, metric, auction_mode='FPA'):
    # if metric == 'RMSE_T':
    study = optuna.create_study(
        direction='maximize' if metric == 'MCR' else 'minimize'
    )
    # elif metric == 'MCR':
    #     study = optuna.create_study(direction='maximize')
    study.optimize(partial(objective_tapid, metric=metric, auction_mode=auction_mode), n_trials=n_trials)

    print('Best trial:')
    trial = study.best_trial
    print(f'  Value: {trial.value}')
    print('  Params: ')
    dict_path = f'best_params/ta_pid_{metric.lower()}_{auction_mode}.pkl'
    params_dict = dict()
    for key, value in trial.params.items():
        print(f'    {key}: {value}')
        params_dict[key] = value
    # save the best params to the file
    with open(dict_path, 'wb') as f:
        pickle.dump(params_dict, f)
    return study

In [ ]:
n = 10
metric = 'RMSE_T' # for Mean Click Ratio use 'MCR'
study_ta_pid = opt_search_tapid(n, metric, auction_mode)

In [ ]:
n = 10
metric = 'CPC_REL' # for Mean Click Ratio use 'MCR'
study_ta_pid = opt_search_tapid(n, metric, auction_mode)

In [ ]:
n = 10
metric = 'MCR' # for Mean Click Ratio use 'MCR'
study_linear = opt_search_tapid(n, metric, auction_mode)

In [ ]:
best_metrics

## M-PID Bidder

In [ ]:
def objective_mpid(trial, metric='RMSE_T', auction_mode='FPA'):
    k_p1 = trial.suggest_float('k_p1', 1e-4, 1, log=True)
    k_p2 = trial.suggest_float('k_p2', 1e-4, 1, log=True)
    k_i1 = trial.suggest_float('k_i1', 1e-4, 1, log=True)
    k_i2 = trial.suggest_float('k_i2', 1e-4, 1, log=True)
    k_d1 = trial.suggest_float('k_d1', 1e-4, 1, log=True)
    k_d2 = trial.suggest_float('k_d2', 1e-4, 1, log=True)
    alpha = trial.suggest_float('alpha', 0.1, 0.9, log=True)
    beta = trial.suggest_float('beta', 0.1, 0.9, log=True)
    coef = trial.suggest_float('coef', 0.1, 0.9, log=True) # Cold start coefficient
    lower_clip = trial.suggest_float('lower_clip', 0.1, 0.9, log=True)
    upper_clip = trial.suggest_float('upper_clip', 1.5, 10, log=True)
    bid_factor = trial.suggest_float('bid_factor', 0.5, 20.0, log=True)
    temp_coef = 1.0
    
    k_dict = {
        'k_p': (k_p1, k_p2),
        'k_i': (k_i1, k_i2),
        'k_d': (k_d1, k_d2)
    }
   
    res = autobidder_check(
        bidder=MPIDBidder,
        params={
            "input_campaigns": campaigns_path,
            "input_stats": stats_path,
            "k_dict": k_dict, 
            "correction": [alpha, beta],
            'cold_start_coef': coef,
            'lower_clip': lower_clip, 
            'upper_clip': upper_clip, 
            'bid_factor': bid_factor,
            #'temp_constr': temp_constr,
            'temp_coef': temp_coef},
        auction_mode=auction_mode,
    )
    print(f"rel_cpc: {res['score'][0]}, rmse: {res['score'][1]}, SCR: {res['score'][2]}")
    if metric == 'RMSE_T':
        best_metrics[2, 0] = min(best_metrics[2, 0], res['score'][1])
        return res['score'][1] 
    elif metric == 'CPC_REL':
        best_metrics[2, 1] = min(best_metrics[2, 1], res['score'][0])
        return res['score'][0]
    elif metric == 'weighted_sum':
        best_metrics[2, 2] = min(best_metrics[2, 2], res['score'][2])
        return res['score'][2]

def opt_search_mpid(n_trials, metric='RMSE_T', auction_mode='FPA'):
    study = optuna.create_study(direction='maximize' if metric == 'MCR' else 'minimize')
    study.optimize(partial(objective_mpid, metric=metric, auction_mode=auction_mode), n_trials=n_trials, n_jobs=1)
    print('Best trial:')
    trial = study.best_trial
    print(f'  Value: {trial.value}')
    print('  Params: ')
    dict_path = f'best_params/m_pid_{metric.lower()}_{auction_mode}.pkl'
    params_dict = dict()
    for key, value in trial.params.items():
        print(f'    {key}: {value}')
        params_dict[key] = value
    # save the best params to the file
    with open(dict_path, 'wb') as f:
        pickle.dump(params_dict, f)
    return study

In [ ]:
def objective_mpid(trial, metric='RMSE_T', auction_mode='FPA'):
    k_p1 = trial.suggest_float('k_p1', 1e-4, 1, log=True)
    k_p2 = trial.suggest_float('k_p2', 1e-4, 1, log=True)
    k_i1 = trial.suggest_float('k_i1', 1e-4, 1, log=True)
    k_i2 = trial.suggest_float('k_i2', 1e-4, 1, log=True)
    k_d1 = trial.suggest_float('k_d1', 1e-4, 1, log=True)
    k_d2 = trial.suggest_float('k_d2', 1e-4, 1, log=True)
    alpha = trial.suggest_float('alpha', 0.1, 0.9, log=True)
    beta = trial.suggest_float('beta', 0.1, 0.9, log=True)
    coef = trial.suggest_float('coef', 0.1, 0.9, log=True) # Cold start coefficient
    lower_clip = trial.suggest_float('lower_clip', 0.1, 0.9, log=True)
    upper_clip = trial.suggest_float('upper_clip', 1.5, 10, log=True)
    bid_factor = trial.suggest_float('bid_factor', 0.5, 20.0, log=True)
    
    k_dict = {
        'k_p': (k_p1, k_p2),
        'k_i': (k_i1, k_i2),
        'k_d': (k_d1, k_d2)
    }
   
    res = autobidder_check(
        bidder=MPIDBidder,
        params={
            "input_campaigns": campaigns_path,
            "input_stats": stats_path,
            "k_dict": k_dict, 
            "correction": [alpha, beta],
            'cold_start_coef': coef,
            'lower_clip': lower_clip, 
            'upper_clip': upper_clip, 
            'bid_factor': bid_factor},
        auction_mode=auction_mode,
    )
    
    print(f"CPC_REL: {res['score'][0]}, rmse: {res['score'][1]}, SCR: {res['score'][2]}")
    
    if metric == 'RMSE_T':
        best_metrics[2, 0] = min(best_metrics[2, 0], res['score'][1])
        return res['score'][1] 
    elif metric == 'CPC_REL':
       
        
        best_metrics[2, 1] = min(best_metrics[2, 1], res['score'][0])
        return res['score'][0] 
    elif metric == 'MCR':
        best_metrics[2, 2] = max(best_metrics[2, 2], res['score'][2])
        return res['score'][2]

def opt_search_mpid(n_trials, metric='RMSE_T', auction_mode='FPA'):
    # if metric == 'RMSE_T':
    study = optuna.create_study(direction='maximize' if (metric == 'MCR') else 'minimize')
    # elif metric == 'MCR':
    #     study = optuna.create_study(direction='maximize')
    study.optimize(partial(objective_mpid, metric=metric, auction_mode=auction_mode), n_trials=n_trials, n_jobs=6)

    print('Best trial:')
    trial = study.best_trial
    print(f'  Value: {trial.value}')
    print('  Params: ')
    dict_path = f'best_params/m_pid_{metric.lower()}_{auction_mode}.pkl'
    params_dict = dict()
    for key, value in trial.params.items():
        print(f'    {key}: {value}')
        params_dict[key] = value
    # save the best params to the file
    with open(dict_path, 'wb') as f:
        pickle.dump(params_dict, f)
    return study

In [ ]:
n = 10
metric = 'CPC_REL' # for Mean Click Ratio use 'MCR'
study_m_pid = opt_search_mpid(n, metric, auction_mode)

In [ ]:
n = 10
metric = 'MCR' # for Mean Click Ratio use 'MCR'
study_m_pid = opt_search_mpid(n, metric, auction_mode)

In [ ]:
n = 10
metric = 'RMSE_T' # for Mean Click Ratio use 'MCR'
study_m_pid = opt_search_mpid(n, metric, auction_mode)

In [ ]:
print('VCG, RMSE_T| REL_CPC| RMSE_T+REL_CPC: ')
np.set_printoptions(precision=5,suppress=True)
print('Linear', best_metrics[0])
print('TA-PID', best_metrics[1])
print('M-PID', best_metrics[2])

In [ ]:
print('FPA, RMSE_T| REL_CPC| RMSE_T+REL_CPC: ')
np.set_printoptions(precision=5,suppress=True)
print('Linear', best_metrics[0])
print('TA-PID', best_metrics[1])
print('M-PID', best_metrics[2])

# Mystique bidder

In [ ]:
def objective_mystique(trial, metric='RMSE_T', auction_mode='FPA'):
   

    pf0 = trial.suggest_float('pf0', 1, 200, log=True)
    C_max = trial.suggest_float('C_max', 1, 100, log=True)
    C_min = trial.suggest_float('C_min', 1e-4, 1, log=True)
    E_max = trial.suggest_float('E_max', 1, 100, log=True)
    E_gmc = trial.suggest_float('E_gmc', 1, 100, log=True)
   
    res = autobidder_check(
        bidder=Mystique,
        params={
            "input_campaigns": campaigns_path,
            "input_stats": stats_path,
            "pf0": pf0, 
            "C_max": C_max,
            'C_min': C_min,
            'E_max': E_max, 
            'E_gmc': E_gmc},
        auction_mode=auction_mode,
    )

    print(f"CPC_REL: {res['score'][0]}, rmse: {res['score'][1]}, SCR: {res['score'][2]}")
    if metric == 'RMSE_T':
        return res['score'][1] 
    elif metric == 'CPC_REL':
        return res['score'][0] 
    elif metric == 'MCR':
        return res['score'][2] 

def opt_search_mystique(n_trials, metric='RMSE_T', auction_mode='FPA'): 
    study = optuna.create_study(direction='maximize' if metric == 'MCR' else 'minimize')
    study.optimize(partial(objective_mystique, metric=metric, auction_mode=auction_mode), n_trials=n_trials, n_jobs=6)
    print('Best trial:')
    trial = study.best_trial
    print(f'  Value: {trial.value}')
    print('  Params: ')
    dict_path = f'best_params/mystique_{metric.lower()}_{auction_mode}.pkl'
    params_dict = dict()
    for key, value in trial.params.items():
        print(f'    {key}: {value}')
        params_dict[key] = value
    # save the best params to file
    with open(dict_path, 'wb') as f:
        pickle.dump(params_dict, f)
    return study

In [ ]:
n = 10
metric = 'RMSE_T' # for Mean Click Ratio use 'MCR'
study_mystique = opt_search_mystique(n, metric,  auction_mode)

In [ ]:
n = 10
metric = 'CPC_REL' # for Mean Click Ratio use 'MCR'
study_mystique = opt_search_mystique(n, metric,  auction_mode)

In [ ]:
n = 10
metric = 'MCR' # for Mean Click Ratio use 'MCR'
study_mystique = opt_search_mystique(n, metric, auction_mode)

### BROI

In [ ]:
def objective_broi(trial, metric='RMSE_T', auction_mode='FPA'):
    ro = trial.suggest_float('ro', 0.01, 200, log=True)
    v_bar = trial.suggest_float('v_bar', 0.01, 200, log=True)
   
    res = autobidder_check(
        bidder=BROI,
        params={
            "input_campaigns": campaigns_path,
            "input_stats": stats_path,
            'ro': ro,
            'v_bar': v_bar
        },
        auction_mode=auction_mode,
    )
    print(f"CPC_REL: {res['score'][0]}, rmse: {res['score'][1]}, SCR: {res['score'][2]}")
    if metric == 'RMSE_T':
        best_metrics[0, 0] = min(best_metrics[0, 0], res['score'][1])
        return res['score'][1] 
    elif metric == 'CPC_REL':
        best_metrics[0, 1] = max(best_metrics[0, 1], res['score'][0])
        return res['score'][0] 
    elif metric == 'MCR':
        best_metrics[0, 2] = max(best_metrics[0, 2], res['score'][2])
        return res['score'][2]
    
def opt_search_broi(n_trials=100, metric='RMSE_T', auction_mode='FPA'):
    # Create pruner
    pruner = optuna.pruners.MedianPruner(
        n_startup_trials=5,
        n_warmup_steps=10,
        interval_steps=1
    )
    # Create study with pruner
    study = optuna.create_study(
        direction='maximize' if (metric == 'MCR') else 'minimize',
        pruner=pruner,
        storage=f'sqlite:///broi_{metric}.db'
    )
    # Optimization
    study.optimize(
        partial(objective_broi, metric=metric, auction_mode=auction_mode),
        n_trials=n_trials,
        catch=(Exception,)
    )
    print('Best trial:')
    trial = study.best_trial
    print(f'  Value: {trial.value}')
    print('  Params: ')
    dict_path = f'best_params/broi_{metric.lower()}_{auction_mode}.pkl'
    params_dict = dict()
    for key, value in trial.params.items():
        print(f'    {key}: {value}')
        params_dict[key] = value
    # save the best params to the file
    with open(dict_path, 'wb') as f:
        pickle.dump(params_dict, f)
    return study

In [ ]:
n = 10
metric = 'CPC_REL' # for Mean Click Ratio use 'MCR'
study_linear = opt_search_broi(n, metric, auction_mode)

In [ ]:
n = 10
metric = 'MCR' # for Mean Click Ratio use 'MCR'
study_linear = opt_search_broi(n, metric, auction_mode)



# Evaluation with the best params

In [ ]:
campaigns_path_test = f"../data/{auction_mode.lower()}/campaigns_{auction_mode.lower()}_filtered_test_final.csv"
stats_path_test = f"../data/{auction_mode.lower()}/stats_{auction_mode.lower()}_filtered_test_final.csv"

In [ ]:
best_metrics_eval = np.ones((3, 3)) * 100000

### Linear Bidder

In [ ]:
def eval_linear(params_dict, metric='RMSE_T', auction_mode='FPA'):
    res = autobidder_check(
        bidder=LinearBidder,
        params={
            "input_campaigns": campaigns_path_test,
            "input_stats": stats_path_test,
            "cold_start_coef": params_dict['coef'],
            "lower_clip": params_dict['lower_clip'],
            "upper_clip": params_dict['upper_clip'],
            "factor": params_dict['factor']
        },
        auction_mode=auction_mode,
    )
    print(f"CPC_REL: {res['score'][0]}, rmse: {res['score'][1]}, SCR: {res['score'][2]}")
    if metric == 'RMSE_T':
        best_metrics[0, 0] = min(best_metrics[0, 0], res['score'][1])
        return res['score'][1] 
    elif metric == 'CPC_REL':
        best_metrics[0, 1] = min(best_metrics[0, 1], res['score'][0])
        print(f'PROBLEMS???{abs(1 - res["score"][0])}')
        return res['score'][0] 
    elif metric == 'MCR':
        best_metrics[0, 2] = max(best_metrics[0, 2], res['score'][2])
        return res['score'][2] 
        


In [ ]:
campaigns = pd.read_csv(campaigns_path_test)
stats = pd.read_csv(stats_path_test)

In [ ]:
%%time
metric = 'RMSE_T'
dict_path = f'best_params/linear_{metric.lower()}_{auction_mode}.pkl'
with open(dict_path, 'rb') as f:
    params = pickle.load(f)

eval_linear(params, metric, auction_mode)

In [ ]:
%%time
metric = 'CPC_REL'
dict_path = f'best_params/linear_{metric.lower()}_{auction_mode}.pkl'
with open(dict_path, 'rb') as f:
    params = pickle.load(f)

eval_linear(params, metric, auction_mode)

In [ ]:
%%time
metric = 'MCR'
dict_path = f'best_params/linear_{metric.lower()}_{auction_mode}.pkl'
with open(dict_path, 'rb') as f:
    params = pickle.load(f)

eval_linear(params, metric, auction_mode)

### TA-PID

In [ ]:
def eval_tapid(params, metric='RMSE_T', auction_mode='FPA'):
    k_dict = {
        'k_p': params['k_p1'],
        'k_i': params['k_i1'],
        'k_d': params['k_d1']
    }
   
    res = autobidder_check(
        bidder=TAPIDBidder,
        params={
            "input_campaigns": campaigns_path_test,
            "input_stats": stats_path_test,
            "k_dict": k_dict, 
            'cold_start_coef': params['coef'],
        },
        auction_mode=auction_mode,
    )
    print(f"CPC_REL: {res['score'][0]}, rmse: {res['score'][1]}, SCR: {res['score'][2]}")
    if metric == 'RMSE_T':
        best_metrics[0, 0] = min(best_metrics[0, 0], res['score'][1])
        return res['score'][1] 
    elif metric == 'CPC_REL':
        best_metrics[0, 1] = min(best_metrics[0, 1], res['score'][0])
        print(f'PROBLEMS???{abs(1 - res["score"][0])}')
        return res['score'][0] 
    elif metric == 'MCR':
        best_metrics[0, 2] = max(best_metrics[0, 2], res['score'][2])
        return res['score'][2] 

In [ ]:
%%time
metric = 'RMSE_T'
dict_path = f'best_params/ta_pid_{metric.lower()}_{auction_mode}.pkl'
with open(dict_path, 'rb') as f:
    params = pickle.load(f)

eval_tapid(params, metric, auction_mode)

In [ ]:
%%time
metric = 'CPC_REL'
dict_path = f'best_params/ta_pid_{metric.lower()}_{auction_mode}.pkl'
with open(dict_path, 'rb') as f:
    params = pickle.load(f)

eval_tapid(params, metric, auction_mode)

In [ ]:
%%time
metric = 'MCR'
dict_path = f'best_params/ta_pid_{metric.lower()}_{auction_mode}.pkl'
with open(dict_path, 'rb') as f:
    params = pickle.load(f)

eval_tapid(params, metric, auction_mode)

### M-PID

In [ ]:
%%time

params = {
    'k_p1': 0.0009872512245452012,
    'k_p2': 0.18897715248785318,
    'k_i1': 0.9966528621547841,
    'k_i2': 0.0003253464269231336,
    'k_d1': 0.07272030450279275,
    'k_d2': 0.002192881493259258,
    'alpha': 0.7071773265998677,
    'beta': 0.17578463114240267,
    'coef': 0.16586633242779408,
    'lower_clip': 0.23922786884323818,
    'upper_clip': 1.5030018770923117,
    'bid_factor': 18.942084662162305
    
}
with open(dict_path, 'wb') as f:
    pickle.dump(params, f)

In [ ]:
def eval_mpid(params, metric='RMSE_T', auction_mode='FPA'):    
    k_dict = {
        'k_p': (params['k_p1'], params['k_p2']),
        'k_i': (params['k_i1'], params['k_i2']),
        'k_d': (params['k_d1'], params['k_d2'])
    }
   
    res = autobidder_check(
        bidder=MPIDBidder,
        params={
            "input_campaigns": campaigns_path_test,
            "input_stats": stats_path_test,
            "k_dict": k_dict, 
            "correction": [params['alpha'], params['beta']],
            'cold_start_coef': params['coef'],
            'lower_clip': params['lower_clip'], 
            'upper_clip': params['upper_clip'], 
            'bid_factor': params['bid_factor'],},
            #'temp_coef': params['temp_coef'],},
        auction_mode=auction_mode,
    )
    print(f"CPC_REL: {res['score'][0]}, rmse: {res['score'][1]}, SCR: {res['score'][2]}")
    if metric == 'RMSE_T':
        best_metrics[0, 0] = min(best_metrics[0, 0], res['score'][1])
        return res['score'][1] 
    elif metric == 'CPC_REL':
        best_metrics[0, 1] = max(best_metrics[0, 1], res['score'][0])
        
        return res['score'][0] 
    elif metric == 'MCR':
        best_metrics[0, 2] = max(best_metrics[0, 2], res['score'][2])
        return res['score'][2] 



In [ ]:
%%time
metric = 'CPC_REL'
dict_path = f'best_params/m_pid_{metric.lower()}_{auction_mode}.pkl'
with open(dict_path, 'rb') as f:
    params = pickle.load(f)

eval_mpid(params, metric, auction_mode)

In [ ]:
%%time
metric = 'RMSE_T'
dict_path = f'best_params/m_pid_{metric.lower()}_{auction_mode}.pkl'
with open(dict_path, 'rb') as f:
    params = pickle.load(f)

eval_mpid(params, metric, auction_mode)

In [ ]:
%%time
metric = 'MCR'
dict_path = f'best_params/m_pid_{metric.lower()}_{auction_mode}.pkl'
with open(dict_path, 'rb') as f:
    params = pickle.load(f)

eval_mpid(params, metric, auction_mode)

In [ ]:
print('VCG, RMSE_T| REL_CPC| RMSE_T+REL_CPC: ')
np.set_printoptions(precision=5,suppress=True)
print('Linear', best_metrics_eval[0])
print('TA-PID', best_metrics_eval[1])
print('M-PID', best_metrics_eval[2])

In [ ]:
print('FPA, RMSE_T| REL_CPC| RMSE_T+REL_CPC: ')
np.set_printoptions(precision=5,suppress=True)
print('Linear', best_metrics_eval[0])
print('TA-PID', best_metrics_eval[1])
print('M-PID', best_metrics_eval[2])

#__Mystique__



In [ ]:
def eval_mystique(params, metric='RMSE_T', auction_mode='FPA'):
   
   
    res = autobidder_check(
        bidder=Mystique,
       params = {
           "input_campaigns": campaigns_path_test,
            "input_stats": stats_path_test,
            "pf0": params['pf0'],
            "C_max": params['C_max'],
            "C_min": params['C_min'],
            "E_max": params['E_max'],
            "E_gmc": params['E_gmc']
    },
        auction_mode=auction_mode,
    )
    print(f"rel_cpc: {res['score'][0]}, rmse: {res['score'][1]}, mcr: {res['score'][2]}")
    if metric == 'RMSE_T':
        best_metrics_eval[1, 0] = min(best_metrics_eval[1, 0], res['score'][1])
        return res['score'][1]  
    elif metric == 'CPC_REL':
        best_metrics_eval[1, 1] = min(best_metrics_eval[1, 1], res['score'][0])
        return res['score'][0]
    elif metric == 'MCR':
        best_metrics_eval[1, 2] = max(best_metrics_eval[1, 2], res['score'][2])
        return res['score'][2]
    else:
        print('problems')

In [ ]:
%%time
metric = 'RMSE_T'
dict_path = f'best_params/mystique_{metric.lower()}_{auction_mode}.pkl'
with open(dict_path, 'rb') as f:
    params = pickle.load(f)

eval_mystique(params, metric, auction_mode)

In [ ]:
%%time
metric = 'CPC_REL'
dict_path = f'best_params/mystique_{metric.lower()}_{auction_mode}.pkl'
with open(dict_path, 'rb') as f:
    params = pickle.load(f)

eval_mystique(params, metric, auction_mode)

In [ ]:
%%time
metric = 'MCR'
dict_path = f'best_params/mystique_{metric.lower()}_{auction_mode}.pkl'
with open(dict_path, 'rb') as f:
    params = pickle.load(f)

eval_mystique(params, metric, auction_mode)

### BROI

In [ ]:
def eval_broi(params, metric='RMSE_T', auction_mode='FPA'):
   
   
    res = autobidder_check(
        bidder=BROI,
       params = {
            "input_campaigns": campaigns_path_test,
            "input_stats": stats_path_test,
            #'theta': params['theta'],
            'ro': params['ro'],
            'v_bar': params['v_bar']
    },
        auction_mode=auction_mode,
    )
    print(f"CPC_REL: {res['score'][0]}, rmse: {res['score'][1]}, SCR: {res['score'][2]}")
    if metric == 'RMSE_T':
        best_metrics[0, 0] = min(best_metrics[0, 0], res['score'][1])
        return res['score'][1] 
    elif metric == 'CPC_REL':
        best_metrics[0, 1] = min(best_metrics[0, 1], res['score'][0])
        
        return res['score'][0] 
    elif metric == 'MCR':
        best_metrics[0, 2] = max(best_metrics[0, 2], res['score'][2])
        return res['score'][2] 

In [ ]:
%%time
metric = 'MCR'
dict_path = f'best_params/broi_{metric.lower()}_{auction_mode}.pkl'
with open(dict_path, 'rb') as f:
    params = pickle.load(f)

eval_broi(params, metric, auction_mode)

In [ ]:
%%time
metric = 'CPC_REL'
dict_path = f'best_params/broi_{metric.lower()}_{auction_mode}.pkl'
with open(dict_path, 'rb') as f:
    params = pickle.load(f)

eval_broi(params, metric, auction_mode)